In [1]:
%matplotlib inline

import os
import numpy as np
import data_loader
import module
from data_loader import DataLoader

import matplotlib.pyplot as plt

import mxnet as mx
from mxnet import nd, autograd, gluon

ctx = mx.cpu()
data_ctx = ctx
model_ctx = ctx

/home/harsha/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dl = DataLoader()

class CustomDataset:
    
    def __init__(self, mode, dataset = 'all'):
        self.x, self.y = dl.load_data(mode, dataset)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return len(self.y)        

In [3]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        data = mx.ndarray.cast(data, dtype='float32')
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [4]:
batch_size = 64

epochs = 5


In [5]:
layer = [1024, 512, 256]
lout = 10
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(layer[0], activation="relu"))
    net.add(gluon.nn.Dense(layer[1], activation="relu"))
    net.add(gluon.nn.Dense(layer[2], activation="relu"))
    net.add(gluon.nn.Dense(lout))

In [ ]:
test_data = mx.gluon.data.DataLoader(CustomDataset('test'), batch_size, last_batch='keep', shuffle=False)
num_examples = len(test_data)e

In [6]:
filename = os.path.join('weights','a2.params')
if not (os.path.isfile(filename)):
    print('No data')
else:
    net.load_parameters(filename, ctx=ctx)

In [7]:
print(net[0].weight.data())


[[-0.09292752  0.05672959 -0.01391952 ...  0.05448012 -0.10445656
   0.02179773]
 [ 0.04634572 -0.0366332  -0.02660666 ... -0.12686409 -0.04906755
  -0.01217611]
 [ 0.00463387 -0.02103078 -0.01972455 ...  0.09320753  0.0850344
   0.02394087]
 ...
 [-0.01200984 -0.02932502 -0.01545437 ... -0.01229158 -0.09649253
   0.00595577]
 [-0.09546756  0.05183284  0.04238009 ... -0.0927762  -0.09471682
   0.01009668]
 [-0.00258917  0.01487542 -0.01414191 ...  0.03011474 -0.02583131
   0.08994317]]
<NDArray 1024x784 @cpu(0)>
